<a href="https://colab.research.google.com/github/Bogula/KISS/blob/main/SimpleChatBotKeras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Einfaches Chatbot Beispiel
#mit Keras

In [ ]:
!pip install keras_visualizer

In [ ]:
# Libraries laden
import json
import string
import random 

import numpy as np


import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)



#Sprachlibraries
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download("punkt")
nltk.download("wordnet")

In [ ]:
# Dies ist ein sogenanntes Dictionary
# es enthält für jeden Intent Beispiele für Eingabetext und Ausgabetext

#data = {"intents": -> "tag" "patterns" "responses"}

data = {"intents": [
             {"tag": "greeting",
              "patterns": ["Hello", "How are you?", "Hi there", "Hi", "Whats up"],
              "responses": ["Howdy Partner!", "Hello", "How are you doing?", "Greetings!", "How do you do?"],
             },
             {"tag": "age",
              "patterns": ["how old are you?", "when is your birthday?", "when was you born?"],
              "responses": ["I am 24 years old", "I was born in 1996", "My birthday is July 3rd and I was born in 1996", "03/07/1996"]
             },
             {"tag": "date",
              "patterns": ["what are you doing this weekend?",
"do you want to hang out some time?", "what are your plans for this week"],
              "responses": ["I am available all week", "I don't have any plans", "I am not busy"]
             },
             {"tag": "name",
              "patterns": ["what's your name?", "what are you called?", "who are you?"],
              "responses": ["My name is Kippi", "I'm Kippi", "Kippi"]
             },
             {"tag": "goodbye",
              "patterns": [ "bye", "g2g", "see ya", "adios", "cya"],
              "responses": ["It was nice speaking to you", "See you later", "Speak soon!"]
             }

]}

In [ ]:
#In die Datenstruktur mal reinschauen?
print(data["intents"][1]["tag"])

In [ ]:
# lemmatizer macht aus Wörtern Wortstämme "did, does" -> "do"
lemmatizer = WordNetLemmatizer()

# Each list to create
words = []
classes = []
doc_X = []
doc_y = []


# Durch alle Intents durchgehen
# tokenize each pattern and append tokens to words, the patterns and
# the associated tag to their associated list

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # add the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])


# reduziere alle Wörter in words und setze die in Kleinbuchstaben
# dabei ignoriere Zeichensetzung
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
print(words)
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

#Am Ende dieses Prozesses haben wir eine Liste aller Wörter
#und der Klassen

In [ ]:
print(classes)

In [ ]:
# list for training data
training = []
out_empty = [0] * len(classes)


# creating the bag of words model
for idx, doc in enumerate(doc_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(doc_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])



In [ ]:
doc_X[2]
#training[0]

In [ ]:
# Durchmischen der Daten und dann ein Array draus machen

random.shuffle(training)
training = np.array(training, dtype=object)

# Features (Inputdaten) und Targets (Zielklassen) den Variablen zuweisen
train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [ ]:
# Länge der Eingabe und Ausgabe Vektoren bestimmen
input_shape = (len(train_X[0]),)
output_shape = len(train_y[0])

#Wie viele Lerndurchläufe wollen wir machen
epochs = 200

# the deep learning model
model = Sequential()
model.add(Dense(128, input_shape=input_shape, activation="relu"))
model.add(Dense(output_shape, activation = "softmax"))

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])

#Wie sieht unser Model aus?
print(model.summary())





In [ ]:
from keras_visualizer import visualizer
visualizer(model, format='png',view=True)
from IPython.display import Image
Image('graph.png')

In [ ]:
history = model.fit(x=train_X, y=train_y, epochs=200, verbose=1)

In [ ]:
def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.2
  y_pred = [[idx, res] for idx, res in enumerate(result) if res > thresh]

  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json): 
  tag = intents_list[0]
  list_of_intents = intents_json["intents"]
  for i in list_of_intents: 
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
# running the chatbot
byetag = False
while  not byetag:
    message = input("")
    intents = pred_class(message, words, classes)
    #print(intents)
    #if intents==["goodbye"]:
    #  break
    result = get_response(intents, data)
    print(result)